##Preparation

In [1]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# imports
import pandas as pd
from sqlalchemy import create_engine

In [3]:
# load table 1
school = pd.read_csv('/content/drive/MyDrive/Coding Dojo/Raw Data/School_Details.csv')

# load table 2
facility = pd.read_csv('/content/drive/MyDrive/Coding Dojo/Raw Data/School_Facility_Details.csv')

##Create SQLite Engine

In [4]:
# create engine
engine = create_engine('sqlite:///school.db', echo = True)

# create connection to engine
conn = engine.connect()

In [5]:
# add dataframe as table in SQLite
sqlite_table = "school_details"
school.to_sql(sqlite_table, conn, if_exists = 'fail')

sqlite_table2 = "school_facility_details"
facility.to_sql(sqlite_table2, conn, if_exists = 'fail')

2022-01-20 23:21:01,152 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("school_details")
2022-01-20 23:21:01,154 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-20 23:21:01,157 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("school_details")
2022-01-20 23:21:01,159 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-20 23:21:01,170 INFO sqlalchemy.engine.Engine 
CREATE TABLE school_details (
	"index" BIGINT, 
	"DistrictName" TEXT, 
	"EstablishmentCode" TEXT, 
	"EstablishmentName" TEXT, 
	"BRCName" TEXT, 
	"CRCName" TEXT, 
	"SchoolLevel" TEXT, 
	"ManagementName" TEXT, 
	"LocalityHabitation" TEXT, 
	"SubdivisionName" TEXT, 
	"ConstituencyName" TEXT, 
	"RevenueBlockName" TEXT, 
	"CivicBodyName" TEXT, 
	"WardName" TEXT, 
	"PostOfficeName" TEXT, 
	"PhoneNo" TEXT, 
	"FaxNo" TEXT, 
	"EmailID" TEXT, 
	"Website" TEXT, 
	"LocatedInRuralOrUrban" TEXT, 
	"DiseCode" TEXT, 
	"EstablishedDate" TEXT, 
	"DistrictHQDistance (in Km)" FLOAT, 
	"CRCDistance (in Km)" FLOAT, 
	"BACDistance (in 

##Write and View SQL Queries

##1. What is the average BACDistance (in Km)? Rename the output column to "avg_bac_distance."

In [6]:
# write sql query 1
sql1 = """
SELECT AVG([BACDistance (in Km)]) AS "avg_bac_distance"
FROM school_details
"""

# read sql query to view results
df1 = pd.read_sql_query(sql1, engine)
df1.head()


2022-01-20 23:21:06,624 INFO sqlalchemy.engine.Engine 
SELECT AVG([BACDistance (in Km)]) AS "avg_bac_distance"
FROM school_details

2022-01-20 23:21:06,628 INFO sqlalchemy.engine.Engine [raw sql] ()


,avg_bac_distance
0,12.681822


##2. What is the average BACDistance (in Km) by School Level? Sort the results from highest average to lowest average.

In [7]:
# write sql query 2
sql2 = """
SELECT AVG([BACDistance (in Km)]), SchoolLevel
FROM school_details
GROUP BY SchoolLevel
ORDER BY AVG([BACDistance (in Km)]) ASC
"""

# read sql query 2 to view results
df2 = pd.read_sql_query(sql2, engine)
df2.head()

2022-01-20 23:21:09,688 INFO sqlalchemy.engine.Engine 
SELECT AVG([BACDistance (in Km)]), SchoolLevel
FROM school_details
GROUP BY SchoolLevel
ORDER BY AVG([BACDistance (in Km)]) ASC

2022-01-20 23:21:09,694 INFO sqlalchemy.engine.Engine [raw sql] ()


,AVG([BACDistance (in Km)]),SchoolLevel
0,NaN,None
1,5.368421,LPS
2,10.863095,SSS
3,12.630872,SS
4,12.830787,PS


##3. Repeat the previous query, but only display results where the average distance is at least 10 km.

In [8]:
# write sql query 3
sql3 = """
SELECT AVG([BACDistance (in Km)]), SchoolLevel
FROM school_details
GROUP BY SchoolLevel
HAVING AVG([BACDistance (in Km)]) >= 10
ORDER BY AVG([BACDistance (in Km)]) ASC
"""

# read sql query 3 to view results
df3 = pd.read_sql_query(sql3, engine)
df3.head()

2022-01-20 23:21:12,693 INFO sqlalchemy.engine.Engine 
SELECT AVG([BACDistance (in Km)]), SchoolLevel
FROM school_details
GROUP BY SchoolLevel
HAVING AVG([BACDistance (in Km)]) >= 10
ORDER BY AVG([BACDistance (in Km)]) ASC

2022-01-20 23:21:12,697 INFO sqlalchemy.engine.Engine [raw sql] ()


,AVG([BACDistance (in Km)]),SchoolLevel
0,10.863095,SSS
1,12.630872,SS
2,12.830787,PS
3,13.692647,JHS


##4. Join the two tables together (school_details and school_facility_details). Start by displaying all of the columns, but limit the resulting rows to just 2.

In [34]:
# write sql query 4
sql4 = """
SELECT *
FROM school_details sd INNER JOIN school_facility_details sfd on sd.EstablishmentName = sfd.EstablishmentName
LIMIT 2
"""

# read sql query 4 to view results
df4 = pd.read_sql_query(sql4, engine)
df4

2022-01-21 00:11:01,711 INFO sqlalchemy.engine.Engine 
SELECT *
FROM school_details sd INNER JOIN school_facility_details sfd on sd.EstablishmentName = sfd.EstablishmentName
LIMIT 2

2022-01-21 00:11:01,715 INFO sqlalchemy.engine.Engine [raw sql] ()


,index,DistrictName,EstablishmentCode,EstablishmentName,BRCName,CRCName,SchoolLevel,ManagementName,LocalityHabitation,SubdivisionName,ConstituencyName,RevenueBlockName,CivicBodyName,WardName,PostOfficeName,PhoneNo,FaxNo,EmailID,Website,LocatedInRuralOrUrban,DiseCode,EstablishedDate,DistrictHQDistance (in Km),CRCDistance (in Km),BACDistance (in Km),IsRecognized,RecognizedBy,ContactPersonName,LowestClass,HighestClass,BoardName,index,EstablishmentCode,EstablishmentName,IsLibraryAvailable,AvailableBooksCount,IsFireExtgAvailable,ExtinguisherCount,HOIRoomAvailable,IsStaffRoomAvailable,NoofStaffRoom,IsLockerForStaffAvailable,IsSportsStoreRoomAvailable
0,0,East,S0169,BUDANG GOVERNMENT PRIMARY SCHOOL (E),Duga Brc,Central Pendam Government Senior Secondary School,PS,State Govt.,BUDANG,Gangtok,Rhenock,Central Pendam,Central Pendam,Sajong,Upper Pendam,None,None,None,None,Urban Area,None,28126,58.0,8.0,16.0,Recognized,HRDD,None,Class 1,Class 5,None,121,S0169,BUDANG GOVERNMENT PRIMARY SCHOOL (E),Available,100,Available,2,Available,Available,1,Not Available,Not Available
1,1,East,S0099,CENTRAL PENDAM GOVERNMENT SENIOR SECONDARY SCHOOL,Duga Brc,Central Pendam Government Senior Secondary School,SSS,State Govt.,CENTRAL PENDAM,Gangtok,West Pandam,Central Pendam,Central Pendam,Karmithang,Upper Pendam,3592263812,None,None,None,Urban Area,800401,15342,50.0,0.0,6.0,Recognized,HRDD,None,Class 1,Class 12,C.B.S.E.,137,S0099,CENTRAL PENDAM GOVERNMENT SENIOR SECONDARY SCHOOL,Available,0,Available,5,Available,Available,2,Not Available,Available


##5. Now, select just the schools where the SchoolLevel is "PS" that do not have libraries.

In [26]:
# write SQL query 5
sql5 = """
SELECT sd.EstablishmentName, sd.SchoolLevel, sfd.IsLibraryAvailable
FROM school_details sd INNER JOIN school_facility_details sfd on sd.EstablishmentName = sfd.EstablishmentName
WHERE sd.SchoolLevel = "PS" AND sfd.IsLibraryAvailable != "Available"
"""

# read sql query 5 to view results
df5 = pd.read_sql_query(sql5, engine)
df5.head()

2022-01-21 00:01:44,745 INFO sqlalchemy.engine.Engine 
SELECT sd.EstablishmentName, sd.SchoolLevel, sfd.IsLibraryAvailable
FROM school_details sd INNER JOIN school_facility_details sfd on sd.EstablishmentName = sfd.EstablishmentName
WHERE sd.SchoolLevel = "PS" AND sfd.IsLibraryAvailable != "Available"

2022-01-21 00:01:44,747 INFO sqlalchemy.engine.Engine [raw sql] ()


,EstablishmentName,SchoolLevel,IsLibraryAvailable
0,EASWARAMMA SAI GURUKOOL ACCADEMY,PS,Not Available
1,NEW GARDEN BOARDING SCHOOL (SAJONG),PS,Not Available
2,"SARASWATI SHISHU VIDHYALAYA, BUDANG",PS,Not Available
3,SAWNEY GOVERNMENT PRIMARY SCHOOL,PS,Not Available
4,UPPER PACHAK GOVERNMENT PRIMARY SCHOOL,PS,Not Available


##6. Build off the previous query, but order the results alphabetically by name.

In [31]:
# write SQL query 6
sql6 = """
SELECT sd.EstablishmentName, sd.SchoolLevel, sfd.IsLibraryAvailable
FROM school_details sd INNER JOIN school_facility_details sfd on sd.EstablishmentName = sfd.EstablishmentName
WHERE sd.SchoolLevel = "PS" AND sfd.IsLibraryAvailable != "Available"
ORDER BY sd.EstablishmentName ASC
"""

# read SQL query 6 to view results
df6 = pd.read_sql_query(sql6, engine)
df6.head()

2022-01-21 00:08:11,265 INFO sqlalchemy.engine.Engine 
SELECT sd.EstablishmentName, sd.SchoolLevel, sfd.IsLibraryAvailable
FROM school_details sd INNER JOIN school_facility_details sfd on sd.EstablishmentName = sfd.EstablishmentName
WHERE sd.SchoolLevel = "PS" AND sfd.IsLibraryAvailable != "Available"
ORDER BY sd.EstablishmentName ASC

2022-01-21 00:08:11,266 INFO sqlalchemy.engine.Engine [raw sql] ()


,EstablishmentName,SchoolLevel,IsLibraryAvailable
0,(I.E.C) INDIAN EVANGALISTIC CRUSADE MISSION PR...,PS,Not Available
1,10TH MILE GOVERNMENT PRIMARY SCHOOL,PS,Not Available
2,AAPGAZI GOVERNMENT PRIMARY SCHOOL,PS,Not Available
3,AARUBOTEY GOVERNMENT PRIMARY SCHOOL,PS,Not Available
4,AHO-YANGTAM GOVERNMENT PRIMARY SCHOOL,PS,Not Available
